In [1]:
# IonPy
import IonPy as ip

# Misc Imports
import yaml

%load_ext yamlmagic

In [22]:
%%yaml default_cfg 

experiment:
  seed: 42
    
dataloader:
  batch_size: 1
  num_workers: 1
  pin_memory: False

optim: 
  _class: torch.optim.Adam
  lr: 1.0e-4
  
train:
  iterations: 100000
  
log:
  checkpoint_freq: 1000 
  root: '?'
  metrics:
    dice_score:
      _fn: pylot.metrics.dice_score
      from_logits: True
      batch_reduction: 'mean' 

data:
  datasets: ["WBC"]
  version: v4.2
  
callbacks:
  epoch:
    - ip.callbacks.PrintLogged
    - ip.callbacks.ETA
    - ip.callbacks.JobProgress
    - ip.callbacks.TerminateOnNaN

<IPython.core.display.Javascript object>

In [ ]:
%%yaml model_cfg  

model:
  _class: universeg.experiment.models.ContextFeatures
  context_filters: [32,32,32,32]
  mix_filters: [32,32,32,32]

In [ ]:
%%yaml dataset_cfg 

data:
  _class: universeg.experiment.datasets.MultiBinarySegment2D
  support_size: '?'

In [3]:
options = {}

In [12]:
from IonPy.util.config import check_missing

def validate_cfg(cfg):

    # Ensure no missing ? markers
    check_missing(cfg)
                  
    return cfg

In [19]:
from IonPy.util import dict_product, Config

base_cfg = Config(default_cfg).update(model_cfg).update(dataset_cfg)

cfgs = []
for cfg_update in dict_product(options):
    cfg = base_cfg.update(cfg_update)
    cfg = validate_cfg(cfg)
    cfgs.append(cfg)

In [1]:
# Get the current remote resources from the CAML cluster
from slurmlite import ExperimentManager 

em = ExperimentManager()

ImportError: cannot import name 'get_remote_resources' from 'IonPy.util' (/Users/victo/Projects/IonPy/util/__init__.py)

In [3]:
# This returns a dataframe with each server and each gpu 
# with th memory which is used.
em.get_remote_resources()

NameError: name 'em' is not defined

In [21]:
import paramiko

# Load YAML configs
with open('config.yml', 'r') as f:
    configs = yaml.safe_load(f)

# Establish SSH connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(configs['remote_host'], username=configs['remote_username'], password=configs['remote_password'])

# Execute machine learning run
cmd = f"python {configs['script']} --config {configs['config_file']} --data {configs['data_dir']}"
stdin, stdout, stderr = ssh.exec_command(cmd)
output = stdout.read().decode()
errors = stderr.read().decode()

# Print output and errors
print(output)
print(errors)

# Close SSH connection
ssh.close()

[None]